In [1]:
import subprocess
from tls_crypt import *

In [2]:
def str2list(s) :
    r = []
    for c in s: r.append(ord(c))
    return r

def list2str(l) :
    r = ''
    for c in l: r += chr(c)
    return r

def hex2list(s):
    assert len(s) % 16 == 0
    l = []
    str = ''
    for c in s:
        str += c
        if len(str) == 2:
            l.append(int(str, 16))
            str = ''
    return l

def hexstr2list(s):
    li = s.split(' ')
    return [int(i, 16) for i in li]

def hexprint(li):
    print('[', end='')
    for i in li : print('{:02x}'.format(i), end=',')
    print('\b]')
    
def list2int(li):
    k = 0;
    for i in li:
        k *= 0x100;
        k += i
    return k

def int2list(k):
    r = []
    while k > 0:
        r.insert(0, k%0x100)
        k = k // 0x100
    return r

In [3]:
aes = AES()
hkdf = HKDF()
x25519 = X25519()

In [65]:
res = subprocess.check_output('./test.sh 2> /dev/null', shell = True).splitlines()

In [66]:
res

[b'F8E1A78594AE7F4EFAD6764B65244FD8EE58F87A536184CC1EB862D93C27C977',
 b'bc0f04232a491458-4b9c0bb24538fb8f',
 b'17058799dfddda71-8905fed9f6d56617',
 b'595db02b0f94bb1b-a6b1a171bd98b854',
 b'6268ec6226a48fc1-4b533fbb4c2c2c1e',
 b'a22043c7d9a8496e-091692707005f255',
 b'057264d87c6d4cb1-b281afbf744a7916',
 b'6330cc8b88deca58-151a2471889a46f7',
 b'2bec3970b663ba98-5ebd5a4ffe9d2116',
 b'fb2964aee83a01a9-24474ed143757afb',
 b'767b71e3afd0330d-c8db5fe85f29458f',
 b'a7b906bfc9fd4a6c-3024bc15232a68eb',
 b'6568BCEBDEB051245D8E5462EB8F2B64D40F59FF0E9135D8BD5BA593DEEDE4C2',
 b'bc0f04232a491458-4b9c0bb24538fb8f',
 b'7565b06b262f7d1a-4e7168cb74a70800',
 b'f71494657d5569ff-b60ae8ee0a359da9',
 b'429a0b3dea7a9338-91562ec0d82c4585',
 b'7b0495b6a3db474f-b47aad381f64dc18',
 b'0b6e0a63b9c9e9af-025f0faa99fa8916',
 b'1b9a1e74d644fc29-f78df42c243ff066',
 b'0591fade8a4f5471-f6ecacb7a2e12c69',
 b'f45155e84f435290-ddfdd675c9ea676c',
 b'858c0c6916388aa6-f96c8a2b6679d201',
 b'ab2118eba4ad60aa-1a95f9edbd1d8967',
 b

In [58]:
a = '16 03 01 02 00 01 00 01 fc 03 03 b6 27 34 3b 77 bf 1f 66 54 f3 d4 68 5a 08 b2 20 75 91 7d 03 e5 1b dc b8 67 06 dc c5 70 be 7e ea 20 d6 1c ac 4b fe ec 6e 41 ea 45 b5 e6 da 53 a2 82 86 f6 0c d7 9f da ad 88 9b ef 09 fd f9 9b 2d b4 00 22 4a 4a 13 01 13 02 13 03 c0 2b c0 2f c0 2c c0 30 cc a9 cc a8 c0 13 c0 14 00 9c 00 9d 00 2f 00 35 00 0a 01 00 01 91 6a 6a 00 00 00 00 00 0e 00 0c 00 00 09 6c 6f 63 61 6c 68 6f 73 74 00 17 00 00 ff 01 00 01 00 00 0a 00 0a 00 08 7a 7a 00 1d 00 17 00 18 00 0b 00 02 01 00 00 23 00 00 00 10 00 0e 00 0c 02 68 32 08 68 74 74 70 2f 31 2e 31 00 05 00 05 01 00 00 00 00 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 12 00 00 00 33 00 2b 00 29 7a 7a 00 01 00 00 1d 00 20'
b = '00 2d 00 02 01 01 00 2b 00 0b 0a 6a 6a 03 04 03 03 03 02 03 01 00 1b 00 03 02 00 02 aa aa 00 01 00 00 15 00 98 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 29 00 33 00 0e 00 08'
c = 'f4 cd f0 05 00 21 20 6e e7 a9 a7 f6 50 90 f9 7e 89 88 a4 71 4b d0 d0 5f d7 a8 cc 96 e4 30 61 81 ac e8 70 70 2d 1a a7'

pri = 0xbff91188283846dd6a2134ef7180ca2b0b14fb10dce707b5098c0dddc813b2df
pub = 0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b
print ('{0:#x}'.format(x25519.mul_g(pri)))

resumption_psk = int(res[0], 16)
ticket_id = res[1].decode()[:-1]
client_hello = hexstr2list(a) + int2list(pub) + hexstr2list(b +' '+ ticket_id +' '+ c)
hkdf.zero_salt()
early = hkdf.extract(int2list(resumption_psk))
hexprint(early)

0xe4ffb68ac05f8d96c99da26698346c6be16482badddafe051a66b4f18d668f0b


ValueError: invalid literal for int() with base 16: '32-a'

In [52]:
client = TLS_client('localhost', 4433)
client.send(client_hello)

r1 = client.recv()
r2 = client.recv()
r3 = client.recv()

In [53]:
server_pub = r1[-38:-6]
ecdhe = x25519.mul(pri, list2int(server_pub))
print('{:x}'.format(ecdhe))

27ccd7f3aa12b9d87d2bd1d589d930ea223eb3302887451ba0e8e8d3c791996b


In [54]:
hexprint(early)
hexprint(r1)

[ec,3f,15,e1,fa,0e,17,bd,b0,62,30,35,16,37,85,13,68,87,34,7b,ec,ed,b8,d2,9f,b8,ea,78,b0,69,60,1d,]
[16,03,03,00,80,02,00,00,7c,03,03,a1,2b,19,d0,9c,66,98,52,cf,64,10,53,7c,85,0b,86,05,9e,71,7f,c0,39,95,55,60,35,63,08,f3,48,6b,a5,20,d6,1c,ac,4b,fe,ec,6e,41,ea,45,b5,e6,da,53,a2,82,86,f6,0c,d7,9f,da,ad,88,9b,ef,09,fd,f9,9b,2d,b4,13,01,00,00,34,00,2b,00,02,03,04,00,33,00,24,00,1d,00,20,49,2e,2b,96,f1,fa,04,ec,e3,fe,2b,a4,50,f5,7f,34,9d,b7,42,19,31,fb,6e,e4,7e,da,80,45,76,a7,ee,3b,00,29,00,02,00,00,]


In [55]:
hkdf.salt(early)
tmp = hkdf.derive_secret('derived', [])
hkdf.salt(tmp)
handshake_secret = hkdf.extract(int2list(ecdhe))
hexprint(handshake_secret)#ok until

hkdf.salt(handshake_secret)
s_hs_traffic = hkdf.derive_secret('s hs traffic', client_hello[5:] + r1[5:])
hexprint(s_hs_traffic)

hkdf.salt(s_hs_traffic)
key = hkdf.expand_label('key', '', 16)
iv = hkdf.expand_label('iv', '', 12)
aes.key(list2int(key))
aes.iv(list2int(iv))
aes.aad(r3[0:5])

r = aes.decrypt(r3[5:-16])
hexprint(r[0])
hexprint(r[1])
hexprint(r3[-16:])

[1f,d6,80,9c,34,c8,44,dd,13,4c,58,84,71,54,33,80,ac,db,26,3b,9b,72,67,4a,31,c1,16,9b,a0,65,13,40,]
[15,9b,1a,41,a8,63,29,2f,8b,ca,08,35,89,cf,af,8f,5f,fe,8b,c4,5a,67,eb,84,ac,76,90,cf,93,06,72,f9,]
[08,00,00,0a,00,08,00,0a,00,04,00,1d,00,17,14,00,00,20,c8,b5,81,a4,49,c8,e6,66,7e,7e,c7,d0,ea,59,d1,3b,e6,0d,6c,e9,82,9c,29,9d,60,c7,08,b1,f5,49,73,7a,16,]
[08,dc,fe,bc,25,a4,77,be,c1,27,fc,d2,0d,2b,78,40,]
[08,dc,fe,bc,25,a4,77,be,c1,27,fc,d2,0d,2b,78,40,]


In [56]:
hexprint(key)
hexprint(iv)
hexprint(handshake_secret)
hexprint(client_hello + r1)

[51,ea,06,29,15,93,db,d8,03,18,e5,d3,f3,f1,95,90,]
[38,69,80,71,d5,d4,c1,da,c4,ca,6d,d1,]
[1f,d6,80,9c,34,c8,44,dd,13,4c,58,84,71,54,33,80,ac,db,26,3b,9b,72,67,4a,31,c1,16,9b,a0,65,13,40,]
[16,03,01,02,00,01,00,01,fc,03,03,b6,27,34,3b,77,bf,1f,66,54,f3,d4,68,5a,08,b2,20,75,91,7d,03,e5,1b,dc,b8,67,06,dc,c5,70,be,7e,ea,20,d6,1c,ac,4b,fe,ec,6e,41,ea,45,b5,e6,da,53,a2,82,86,f6,0c,d7,9f,da,ad,88,9b,ef,09,fd,f9,9b,2d,b4,00,22,4a,4a,13,01,13,02,13,03,c0,2b,c0,2f,c0,2c,c0,30,cc,a9,cc,a8,c0,13,c0,14,00,9c,00,9d,00,2f,00,35,00,0a,01,00,01,91,6a,6a,00,00,00,00,00,0e,00,0c,00,00,09,6c,6f,63,61,6c,68,6f,73,74,00,17,00,00,ff,01,00,01,00,00,0a,00,0a,00,08,7a,7a,00,1d,00,17,00,18,00,0b,00,02,01,00,00,23,00,00,00,10,00,0e,00,0c,02,68,32,08,68,74,74,70,2f,31,2e,31,00,05,00,05,01,00,00,00,00,00,0d,00,14,00,12,04,03,08,04,04,01,05,03,08,05,05,01,08,06,06,01,02,01,00,12,00,00,00,33,00,2b,00,29,7a,7a,00,01,00,00,1d,00,20,e4,ff,b6,8a,c0,5f,8d,96,c9,9d,a2,66,98,34,6c,6b,e1,64,82,ba,dd,da,fe,05,1a,66,b4,f1,

In [37]:
hexprint(s_hs_traffic)

[56,f4,80,75,69,e2,e2,7f,f1,55,4c,10,4a,86,0a,7a,a2,de,b7,cc,0e,be,ca,3b,f3,76,e8,8a,62,59,f7,09,]
